# 1. P-hacking

## 1.1 Introduction

This exercise is meant to demonstrate p-hacking, the misuse of data analysis to find patterns in data that can be presented as statistically significant. P-hacking is poor scientific practice, because it ignores the multiple-comparisons problem: the more statistical tests are performed on the data and the more inferences are made, the more likely erroneous inferences (like false positives or Type I errors) become.

This notebook was adapted from https://github.com/jfpuget/p-hacking/blob/master/Green-dice-p-value-hacking.ipynb. It demonstrates how to mislead people about a scientific experiment by hacking p-values following the methodology from this [xkcd comic](http://xkcd.com/882/):

<img src="http://imgs.xkcd.com/comics/significant.png"></img>

Let's start by importing the relevant libraries.

In [1]:
# Import libraries.
import numpy as np
import pandas as pd
from numpy.random import randint

## 1.2 The experiment

We are given 20 dice colors, and we roll dice for each color 1,000 times.  We report the number of sixes. We set a fixed seed, so the results are reproducible

In [ ]:
# Define the dice colors.
dice = ['Purple', 'Brown', 'Pink', 'Blue', 'Teal', 
        'Salmon', 'Red', 'Turquoise', 'Magenta', 'Yellow', 
        'Grey', 'Tan', 'Cyan', 'Green', 'Mauve',
        'Beige', 'Lilac', 'Black', 'Peach', 'Orange']


# Set a fixed seed, so we all get the same results.
np.random.seed(75000)


# Define a function to run the dice rolling experiments.
def dice_experiments(dice, n):
    df = pd.DataFrame(index = dice)
    for die in dice:
        # Simulate the roll of a die n times.
        result = randint(1, (6 + 1), size = n)

        # Count how many times '6' was the outcome.
        df.loc[die, 'Number of Six'] = np.sum(result[result == 6]) // 6
    return df


# Run the experiment 1,000 times and show the results.
df = dice_experiments(dice, 1000)
df.T

,Purple,Brown,Pink,Blue,Teal,Salmon,Red,Turquoise,Magenta,Yellow,Grey,Tan,Cyan,Green,Mauve,Beige,Lilac,Black,Peach,Orange
Number of Six,151.0,167.0,158.0,167.0,181.0,162.0,170.0,161.0,165.0,180.0,172.0,164.0,181.0,188.0,165.0,172.0,178.0,176.0,173.0,157.0


The green color has 188 times a six, which is rather high. Indeed, average value is 1,000 / 6 = 167.

The probability that green dice have at least 188 sixes is about 4% as we will see below.  This is the *p-value* for the green color result. If we decided to watch the green color before the experiment, then this low p-value would warrant us a scientific publication.  

Let's see how unlikely this result is by running our experiment many times.

In [3]:
def get_fraction_one_color(color, dice, k, n, repeat):
    success = 0.0
    for experiment in range(repeat):
        df = dice_experiments(dice, n)
        if df.loc[color, 'Number of Six'] >= k:
            success += 1
    return success / repeat

get_fraction_one_color('Green', dice, 188, 1000, 1000)

0.041

This is about 4.1%, i.e. rather unlikely. There must be a reason why green dice favor six that much!

## 1.3 Exercise

Do you agree with the analysis? Is a low p-value good enough for this scenario?

Is a low p-value good enough for this scenario?


## 1.4 Solution

We misinterpreted what we did and our conclusion is misleading.  What we did was to run the experiment for 20 colors, then select the color with the highest number of sixes.  The p-value should be the probability that at least one color gets at least 188 sixes. 

Let's evaluate this probability experimentally.

In [4]:
def get_fraction_any_color(dice, k, n, repeat):
    success = 0.0
    for experiment in range(repeat):
        df = dice_experiments(dice, n)
        if df['Number of Six'].max() >= k:
            success += 1
    return success / repeat

get_fraction_any_color(dice, 188, 1000, 1000)

0.556

This probability is about 56%, i.e. very likely.  There is nothing special about our dice after all.

This little code may look contrived, but it is not. Scientists got publications using a similar methodology, see this [blog](https://www.ibm.com/developerworks/community/blogs/jfp/entry/green_dice_are_loaded_welcome_to_p_hacking) for details.

## 1.5 Ultimate p-hacking

We wanted to mimick the xkcd comic, hence we looked for a random seed that yields the result we want.  This is really bad p-hacking, as we define first the p-value, then find experiment settings that produce it. :)

In [5]:
def find_seed(dice, k, n, repeat, rounding):
    nseed = 0.0
    for seed in range(0, repeat, rounding):
        np.random.seed(seed)
        df = dice_experiments(dice, n)
        m = df['Number of Six'].max()
        if m == k and m == df.loc['Green','Number of Six']:
            return seed
    
res = find_seed(dice, 188, 1000, 1000000, 1000)
res

75000